<a href="https://colab.research.google.com/github/souvikmajumder26/Any-Face-Clustering/blob/main/*Any_Face_Clustering_Streamlit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install face_recognition

     |████████████████████████████████| 100.1 MB 19 kB/s 
  Created wheel for face-recognition-models: filename=face_recognition_models-0.3.0-py2.py3-none-any.whl size=100566186 sha256=6be8ea30ca7d40b1a77cf0d0d04c5067e61ce698d6329385ba76dab919c58675
  Stored in directory: /root/.cache/pip/wheels/d6/81/3c/884bcd5e1c120ff548d57c2ecc9ebf3281c9a6f7c0e7e7947a
Successfully built face-recognition-models


In [ ]:
!pip install -q streamlit

     |████████████████████████████████| 9.9 MB 2.7 MB/s 
     |████████████████████████████████| 111 kB 65.3 MB/s 
     |████████████████████████████████| 4.3 MB 39.3 MB/s 
     |████████████████████████████████| 181 kB 36.8 MB/s 
     |████████████████████████████████| 76 kB 4.6 MB/s 
     |████████████████████████████████| 164 kB 42.0 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 130 kB 20.4 MB/s 
     |████████████████████████████████| 793 kB 5.9 MB/s 
     |████████████████████████████████| 380 kB 55.6 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jupyter-console 5.2.0 requires prompt-toolkit<2.0.0,>=1.0.0, but you have prompt-toolkit 3.0.28 which is incompatible.
google-colab 1.0.0 requires ipykernel~=4.10, but you have ipykernel 6.9.2 which is incompatible.
google-colab 1.0.0 requires ipython

In [ ]:
%%writefile streamlit-app.py

import cv2
import face_recognition

import os
import tempfile
import numpy as np
import streamlit as st

from sklearn.cluster import DBSCAN
from imutils import build_montages


st.title("Any Face Clustering")

uploaded_files = st.file_uploader("Upload Images :", accept_multiple_files = True)

try:
  if uploaded_files is not None:
    data = []
    for f in uploaded_files:
      tfile = tempfile.NamedTemporaryFile(delete=False)
      tfile.write(f.read())
      image = cv2.imread(tfile.name)
      rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      #st.write(rgb)
      #st.image(rgb)
      boxes = face_recognition.face_locations(rgb, model = "cnn")
      encodings = face_recognition.face_encodings(rgb, boxes)
      #st.write(boxes)
      #st.write(encodings)
      d = [{"imagePath": tfile.name, "loc": box, "encoding": enc} for (box, enc) in zip(boxes, encodings)]
      data.extend(d)
    # converting the data into a numpy array
    data_arr = np.array(data)
    # extracting the 128-d facial encodings and placing them in a list
    encodings_arr = [item["encoding"] for item in data_arr]

    # initializing and fitting the clustering model on the encoded data
    cluster = DBSCAN().fit(encodings_arr)
    #st.write(data)
    #st.write(data_arr)
    #st.write(np.shape(encodings_arr))
    #st.write(encodings_arr)  
    st.write("Model Fit: SUCCESS")
    #st.write(cluster.n_features_in_)
    #st.write(cluster.labels_)
    
    labelIDs = np.unique(cluster.labels_)
    numUniqueFaces = len(np.where(labelIDs > -1)[0])
    
    st.write("Number of unique faces identified:", numUniqueFaces)
    #st.write(labelIDs)

    # loop over the unique face integers
    for labelID in labelIDs:
      # find all indexes into the 'pkl_data' array that belong to the current label ID, then randomly sample a maximum of 15 indexes from the set
      #print("[INFO] faces for face ID: {}".format(labelID))
      idxs = np.where(cluster.labels_ == labelID)[0]
      idxs = np.random.choice(idxs, size = min(15, len(idxs)), replace = False)
      # initialize the list of faces to include in the montage
      faces = []
      
      for i in idxs:
        # load the input image and extract the face ROI
        current_image = cv2.imread(data_arr[i]["imagePath"])
        rgb_current_image = cv2.cvtColor(current_image, cv2.COLOR_BGR2RGB)
        (top, right, bottom, left) = data_arr[i]["loc"]
        current_face = rgb_current_image[top:bottom, left:right]
        # force resize the face ROI to 96x96 and then add it to the faces montage list
        current_face = cv2.resize(current_face, (96, 96))
        faces.append(current_face)
        
      # create a montage using 96x96 "tiles" with 5 rows and 5 columns
      montage = build_montages(faces, (96, 96), (3, 3))[0]
      
      # show the output montage
      current_title = "Face ID #{}".format(labelID)
      current_title = "Unknown Faces" if labelID == -1 else current_title
      #cv2_imshow(montage)
      st.write(current_title)
      st.image(montage)
  else:
    st.write("Please wait for the images to be uploaded...")
except:
  pass

Writing streamlit-app.py


In [ ]:
!ls

sample_data  streamlit-app.py


In [ ]:
!streamlit run streamlit-app.py & npx localtunnel --port 8501

2022-03-16 21:40:14.443 INFO    numexpr.utils: NumExpr defaulting to 2 threads.

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:8501
  External URL: http://35.237.60.41:8501

npx: installed 22 in 6.144s
your url is: https://nasty-dragonfly-70.loca.lt
2022-03-16 21:40:54.747 Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/streamlit/script_runner.py", line 430, in _run_script
    exec(code, module.__dict__)
  File "/content/streamlit-app.py", line 3, in <module>
    import face_recognition
  File "/usr/local/lib/python3.7/dist-packages/face_recognition/__init__.py", line 7, in <module>
    from .api import load_image_file, face_locations, batch_face_locations, face_landmarks, face_encodings, compare_faces, face_distance
  File "/usr/local/lib/python3.7/dist-packages/face_recognition/api.py", line 26, in <module>
    cnn_face_detector = dlib.cnn_face_detection_model_v1(cnn_face_detection_model)
RuntimeError: Error